In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
import DifferentialEquations as DE
using CairoMakie
using GLMakie

# Harmonic Balance using multiple harmonics

## **Try out different methods to do the Symbolic computations on higher harmonics**

Using Python's famous SymPy

In [ ]:
@time begin
    x = sp.symbols("x")
    f = sp.cos(40x)^4
    
    fs = sp.sympy.fourier_series(f, (x, -sp.PI, sp.PI))
end
println(fs.truncate(n=10))

Using Julia's ApproxFun.jl

In [ ]:
@time begin
    # Define on CosSpace (cosine series only)
    f = AF.Fun(x -> cos(5x)^4, AF.CosSpace())
    
    coeffs = AF.coefficients(f)
end

println("Cosine Series:")
for (k, c) in enumerate(coeffs)
        if abs(c) > 1e-10
            println("$(c) * cos($((k-1))x)")
        end
end

## Use the SymPy method to start a potential Harmonic Balance

$$
u = \sum_{k=1}^H A_k \cos(k\omega t) + B_k \sin(k\omega t)
$$

Define the variable parameters for the wave equation

In [ ]:
gamma = 0;
omega = 3;
gamma3 = 0;
g0 = 9.80665; # m / s^2
height = 5; # m

Define the constants specific to the discretizations

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
Nt = 5
N = 100;
harmonics = 1; # number of harmonics
order = 2;
stepx = (xright-xleft)/N;

In [ ]:
# Define symbolics
Model.@parameters x, t;
u = 0
j = 1
vars = []

bcs = []
Dx = Model.Differential(x);
Dt = Model.Differential(t);

for i in 1:(2*harmonics)
    # Create variable A(..), B(..), ...
    c = Char('A' + i - 1)
    v = eval(:(Model.@variables $(Symbol(c))(..)))
    # Call the function with argument(s) when using it
    if isodd(i)
        u += v[1](x) * sin(j * omega*t)  # v[1](t) instead of v
    else
        u += v[1](x) * cos(j * omega*t)
        j+=1
    end
    
    push!(vars, v[1](x))  # v is a vector, v[1] is the function
        
    push!(bcs, v[1](xleft) ~ 0)
    push!(bcs, v[1](xright) ~ 0)
end

F = 50 * exp(-40*(x^2))*sin(omega*t)

y = Dt(Dt(u)) - 9*Dx(Dx(u)) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - F;
y_exp = Symb.expand(Model.expand_derivatives(y));
symbolics_list = Symb.arguments(y_exp, +);

In [ ]:
y

The assumed ansatz looks the following

In [ ]:
u

While the substituted ansatz inside the PDE is the following:

In [ ]:
y_exp

For example let's take this term, the sixth term in the equation above:

In [ ]:
symbolics_list[2]

Now we need to deal with all the trigonometric term and expand them using Sympy's `fourier_series` method

In [ ]:
println("harmonics = ", harmonics)
println("Number of vars: ", length(vars))
println("vars = ", vars)
println("Number of bcs: ", length(bcs))
println("bcs = ", bcs)

In [ ]:
t_sympy = sp.symbols("t")
x_sympy = sp.symbols("x")
Symb.@variables x t;
finished_terms = []  # Initialize empty list

"""
The idea for this new implementation is to use the form of the PDE with the ansatz substituted in and extract each term (separate wrt addition/subtraction)

Each term is then converted from the Symbolics.jl form to the SymPy form and the part dependent on t is extracted for the Fourier series. The expanded
Fourier Series term is divided by the SymPy non-simplified term. Convert back to Symbolics.jl this form and by multiplying it with the original "term" the 
trigonometric term to the power (e.g. cos(x)^3) is cancelled out.
"""

for (i, term) in enumerate(symbolics_list)
    # Transform term by term into sympy form
    term_sympy = Symb.symbolics_to_sympy(term)
    current_term = term_sympy.as_independent(t_sympy)[2] # extract term sinusoidal term dependent on t
    
    fs = sp.sympy.fourier_series(current_term, (t_sympy, -sp.PI, sp.PI)) # simplify using Sympy's Fourier Series
    finished_sympy_term = fs.truncate() / current_term 
    finished_symb_term = Symb.sympy_to_symbolics(finished_sympy_term, [t])*symbolics_list[i]
    push!(finished_terms, finished_symb_term)

    if i == 2
        println("We start with the following Symbolics.jl term ", symbolics_list[i])
        println("With the sympy fourier series convert ", current_term,  " to ", fs.truncate())
        println("Now write this term as ", finished_sympy_term)
        println("and multiply it with the term from the symbolics.jl form, in such a way to simplify the cubic cosine")
        println(symbolics_list[i], "*",     finished_sympy_term)
    end
    # println(finished_symb_term)

end

In [ ]:
final_expression

In [ ]:
final_expression = Symb.simplify(sum(finished_terms));

eqs = []

for i in 1:harmonics
    sin_coef = Symb.coeff(final_expression, sin(i*omega*t));
    cos_coef = Symb.coeff(final_expression, cos(i*omega*t));
    push!(eqs, sin_coef ~ 0)
    push!(eqs, cos_coef ~ 0)
end

In [ ]:
eqs

In [ ]:
eqs[1]

In [ ]:
eqs[2]

## Solve the nonlinear problem using ModelToolkit.jl and NonlinearSolve

In [ ]:
println("Equations:")
for eq in eqs
    println(eq)
end

In [ ]:
# Define the domain
using MethodOfLines, NonlinearSolve
domains = [x ∈ Interval(xleft, xright)];

# Create the PDESystem
Model.@named pde_system = Model.PDESystem(eqs, bcs, domains, [x], vars);

# Discretization
discretization = MOLFiniteDifference([x => stepx], nothing, approx_order=2);

# Convert to NonlinearProblem
prob = discretize(pde_system, discretization);

# Create new u0
# Correct solution from Python (99 interior points each)
A_interior = [0.0, 0.006833751648467837, 0.013114089594504325, 0.01884754442840567, 0.024044878570996152, 0.028720831659655634, 0.03289377817352499, 0.03658531045208937, 0.03981976276355311, 0.042623693761736615, 0.04502534547321372, 0.04705409687762476, 0.04873992923560242, 0.050112918680542004, 0.0512027693606488, 0.0520383977604134, 0.05264757592280879, 0.05305663831165783, 0.0532902541621095, 0.05337126450815307, 0.05332058076236631, 0.053157139833772696, 0.05289790934900182, 0.052557935600703716, 0.052150426366469514, 0.05168686067818691, 0.051177117914767395, 0.05062961916835569, 0.05005147461866989, 0.049448631566355365, 0.04882601875432899, 0.04818768358549106, 0.04753691977677522, 0.046876383836549365, 0.0462081994903714, 0.04553404979582069, 0.04485525717706568, 0.044172851978304076, 0.04348763039232808, 0.042800202780148594, 0.04211103347583335, 0.041420473184007066, 0.04072878504176009, 0.040036165346515634, 0.03934275985927097, 0.03864867648891388, 0.03795399505615459, 0.03725877473103585, 0.03656305964015086, 0.035866883051193406, 0.035170270464602464, 0.03447324187519479, 0.033775813410465504, 0.03307799850588298, 0.03237980873995432, 0.03168125442191658, 0.0309823450014345, 0.030283089351538034, 0.029583495962198197, 0.028883573071535627, 0.028183328753931713, 0.027482770978649675, 0.02678190764847212, 0.02608074662492721, 0.02537929574459985, 0.0246775628295729, 0.023975555694039547, 0.02327328214844166, 0.022570750002024253, 0.0218679670643853, 0.021164941146394167, 0.02046168006071682, 0.019758191622098388, 0.019054483647497334, 0.01835056395612966, 0.017646440369459095, 0.016942120711155136, 0.01623761280703206, 0.015532924484976856, 0.01482806357487067, 0.0141230379085066, 0.013417855319505283, 0.012712523643229348, 0.012007050716697155, 0.01130144437849614, 0.010595712468695955, 0.009889862828761479, 0.00918390330146574, 0.008477841730802816, 0.007771685961900675, 0.0070654438409340214, 0.006359123215037103, 0.005652731932216523, 0.0049462778412640305, 0.004239768791669306, 0.003533212633532734, 0.002826617217478175, 0.0021199903945657277, 0.0014133400162044875, 0.0007066739340653, 0.0]

B_interior = [0.0, -1.519687349952541e-7, -3.039224840612404e-7, -4.5577888318775173e-7, -6.073512509415176e-7, -7.583376330445449e-7, -9.083284468039313e-7, -1.0568262059772658e-6, -1.2032718523397737e-6, -1.3470733476099088e-6, -1.487633298593821e-6, -1.6243734261276623e-6, -1.7567545920294184e-6, -1.884291836958455e-6, -2.0065644451316904e-6, -2.1232214452893194e-6, -2.2339831993922146e-6, -2.338639847000131e-6, -2.4370473929883567e-6, -2.529122177674596e-6, -2.614834377100724e-6, -2.6942010683053935e-6, -2.7672792761167728e-6, -2.834159305605525e-6, -2.8949585648824506e-6, -2.9498159999319317e-6, -2.9988871975673073e-6, -3.0423401635325234e-6, -3.0803517483485757e-6, -3.1131046712967245e-6, -3.1407850804146617e-6, -3.1635805811741193e-6, -3.181678666519279e-6, -3.1952654844414197e-6, -3.204524884876683e-6, -3.209637694406806e-6, -3.2107811742736106e-6, -3.208128624081697e-6, -3.201849099942037e-6, -3.192107221524557e-6, -3.179063047464738e-6, -3.1628720028017066e-6, -3.1436848456523844e-6, -3.1216476632132237e-6, -3.096901889506169e-6, -3.069584339130217e-6, -3.03982725272326e-6, -3.0077583509532536e-6, -2.973500894707255e-6, -2.9371737497866115e-6, -2.8988914548926313e-6, -2.8587642920372336e-6, -2.8168983587677243e-6, -2.7733956417779817e-6, -2.728354091608584e-6, -2.6818676982300347e-6, -2.634026567367038e-6, -2.584916997465657e-6, -2.534621557235119e-6, -2.483219163716171e-6, -2.4307851608413325e-6, -2.37739139846125e-6, -2.323106311817132e-6, -2.2679950014429725e-6, -2.2121193134836304e-6, -2.1555379204163284e-6, -2.0983064021640487e-6, -2.040477327589857e-6, -1.982100336361517e-6, -1.9232222211759445e-6, -1.8638870103331386e-6, -1.8041360506492803e-6, -1.7440080906986968e-6, -1.6835393643743868e-6, -1.6227636747567791e-6, -1.5617124782803833e-6, -1.5004149691879552e-6, -1.4388981642617821e-6, -1.377186987821661e-6, -1.3153043569791178e-6, -1.2532712671373894e-6, -1.1911068777266634e-6, -1.1288285981640528e-6, -1.066452174027751e-6, -1.0039917734347957e-6, -9.414600736118502e-7, -8.788683476483852e-7, -8.162265514216337e-7, -7.535434106826642e-7, -6.908265082929107e-7, -6.280823716004736e-7, -5.653165599454973e-7, -5.025337522839147e-7, -4.3973783491883717e-7, -3.7693198932885513e-7, -3.1411878008250967e-7, -2.5130024282818007e-7, -1.8847797234863048e-7, -1.2565321066944798e-7, -6.28269352106008e-8, 0.0]

# Check the length of prob.u0
println("Length of prob.u0: ", (prob.u0))
println(prob.f)
println("Expected: 2 * 99 = 198 (for 99 interior points, A and B)")

n = length(prob.u0) ÷ 2
println("n = ", n)

# Build u0: [A₁, A₂, ..., Aₙ, B₁, B₂, ..., Bₙ]
u0_new = vcat(A_interior[1:n], B_interior[1:n])

# Remake and solve
prob_new = remake(prob, u0=u0_new)
sol = NonlinearSolve.solve(prob_new, NewtonRaphson(), reltol=1e-5, abstol=1e-5)

In [ ]:
# See what the discretized problem looks like
println("Variables in discretized problem:")
println(prob.f.syms)  # or prob.f.sys if available

# Or check the full structure
println(keys(prob.f))

Once the system of differential equations is solved, extract the necessary coefficients

In [ ]:
# extract solution
solution_coeffs = []
for i in 1:(2*harmonics)
    # Extract solution    
    push!(solution_coeffs, sol[vars[i]]);
end

In [ ]:
print(solution_coeffs[1])

## Plot the results

In [ ]:
dt = 0.01
xgrid = collect(range(start=0.0, stop=1.0, step=stepx))

# Observable for u(x,t)
# Initialize observable with an array of zeros
u = Observable(zeros(length(xgrid)))

fig = Figure(resolution = (800, 500))
ax = Axis(fig[1, 1],
    title = "t = 0.0",
    xlabel = "x",
    ylabel = "u(x, t)",
    limits = (nothing, nothing, -0.5, 0.5)
)
lines!(ax, xgrid, u)
display(fig)

@async begin
    for k in 1:Int(Nt/dt)
        t_discrete = k * dt
        
        # Reset and compute the full sum
        u_new = zeros(length(xgrid))
        j = 1
        for i in 1:(2*harmonics)
            if isodd(i)
                u_new .+= solution_coeffs[i] .* sin(j * omega * t_discrete)
            else
                u_new .+= solution_coeffs[i] .* cos(j * omega * t_discrete)
                j += 1
            end
        end
        
        # Single assignment triggers the update
        u[] = u_new
        ax.title = "t = $(round(t_discrete, digits=2))"
        sleep(1/10)
    end
end

In [ ]:
using FFTW;

u_idx_Fourier = Float64[]
space_idx = 50

dt_fourier = 1/500
for k in 0:dt_fourier:Nt - dt_fourier
    u_Fourier = 0.0
    j = 1
    for i in 1:(2*harmonics)
        # println(i)
        if isodd(i)
            u_Fourier += solution_coeffs[i][space_idx] .* sin(j * omega * k)
        else
            u_Fourier += solution_coeffs[i][space_idx] .* cos(j * omega * k)
            j += 1
        end       
    end
    push!(u_idx_Fourier, u_Fourier)
    # push!(u_idx_Fourier, solution_coeffs[1][space_idx].* sin(1 * omega * k) + solution_coeffs[2][space_idx].* cos(1 * omega * k))
end

fs_time = 1/dt_fourier
N = length(u_idx_Fourier)

F = fftshift(fft(u_idx_Fourier))
freqs = fftshift(fftfreq(N, fs_time))

# Plotting with Makie
f = Figure()
ax2 = Axis(f[1, 2],
    xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
)
ax1 =Axis(f[1, 1],
    ylabel = "Amplitude (m)",
    xlabel = "Time (s)",
)
peak_freq = freqs[argmax(abs.(F))]
vline_positions = [-peak_freq, peak_freq]

# Correct normalization: divide by N, not fs_time
lines!(ax1, 0:dt_fourier:(Nt -dt_fourier), u_idx_Fourier)
lines!(ax2, freqs, abs.(F) ./ Nt)
xlims!(ax2, -20, 20)
f